In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
from concurrent.futures import ThreadPoolExecutor
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
from pathlib import Path
import os

In [ ]:
# Diretório base onde os arquivos CSV estão localizados
base_diretório = Path(os.getcwd())
diretório_princapl = base_diretório.parent
caminho_base = diretório_princapl / 'Bases'
caminho_arquivos = diretório_princapl / 'data'

# Definir caminho dos arquivos 
arquivos = {
    'Bitcoin': caminho_base / 'Bitcoin Historical Data.csv',
    'Ethereum': caminho_base / 'Ethereum Historical Data.csv',
    'BNB': caminho_base / 'BNB Historical Data.csv',
    'Solana': caminho_base / 'Solana Historical Data.csv',
}

# Ler o arquivo ano.txt e pegar o ano
with open(diretório_princapl/'data'/'ano.txt', 'r') as file:
    ano = int(file.read().strip())

# Função para tratar os dados de cada criptomoeda
def tratar_dados_cripto(caminho_csv):
    # Ler o arquivo CSV
    acao = pd.read_csv(caminho_csv)
    acao['Date'] = pd.to_datetime(acao['Date'])

    return acao

# Aplicar a função para cada criptomoeda e armazenar o resultado em um DataFrame
df_bitcoin = tratar_dados_cripto(arquivos['Bitcoin'])
df_ethereum = tratar_dados_cripto(arquivos['Ethereum'])
df_bnb = tratar_dados_cripto(arquivos['BNB'])
df_solana = tratar_dados_cripto(arquivos['Solana'])


def previsao(acao, nome):
    # Separar um df no qual vai conter apenas os dado para treinamento
    # Que é com base no ano escolhido pelo jogador
    df_treinamento = acao[acao['Date'].dt.year < ano]

    # Apenas cotação dos dados de treinamento 
    cotacao_treinamento = df_treinamento['Price'].to_numpy().reshape(-1, 1)

    # Cotação dos dados da base toda
    cotacao = acao['Price'].to_numpy().reshape(-1, 1)

    # Armazenar tamanho dos dados de treinamento
    tamanho_dados_treinamento = int(len(cotacao_treinamento) * 1)

    #escalar os dados entre 0 e 1, para deixar mais fácil o processamento
    #dados em escala pré definidas são mais fáceis de lidar. 
    escalador = MinMaxScaler(feature_range=(0, 1))

    dados_entre_0_e_1_treinamento = escalador.fit_transform(cotacao[0: tamanho_dados_treinamento, :])
    dados_entre_0_e_1_teste = escalador.transform(cotacao[tamanho_dados_treinamento: , :])

    dados_entre_0_e_1 = list(dados_entre_0_e_1_treinamento.reshape(
        len(dados_entre_0_e_1_treinamento))) + list(dados_entre_0_e_1_teste.reshape(len(dados_entre_0_e_1_teste)))
                                                    
    dados_entre_0_e_1 = np.array(dados_entre_0_e_1).reshape(len(dados_entre_0_e_1), 1)
    dados_para_treinamento = dados_entre_0_e_1[0: tamanho_dados_treinamento, :]

    #dados que serão usados para gerar o resultado
    treinamento_x = []
    #cotação que aconteceu de fato
    treinamento_y = []

    for i in range(60, len(dados_para_treinamento)):

        #60 ultimos dias
        treinamento_x.append(dados_para_treinamento[i - 60: i, 0])
        #cotacao
        treinamento_y.append(dados_para_treinamento[i, 0])

        if i <= 61:
            print(treinamento_x)
            print(treinamento_y)
    
    #transformando as listas em arrays e dando reshape 3d 

    treinamento_x, treinamento_y = np.array(treinamento_x), np.array(treinamento_y)
    treinamento_x = treinamento_x.reshape(treinamento_x.shape[0], treinamento_x.shape[1], 1)

    #construindo o modelo
    modelo = Sequential()

    modelo.add(LSTM(100, return_sequences=True, input_shape=(treinamento_x.shape[1], 1)))
    modelo.add(LSTM(50, return_sequences=False))
    modelo.add(Dense(25))
    modelo.add(Dense(1))

    treinamento_x.shape[1]

    #copilando o modelo
    modelo.compile(optimizer="adam", loss="mean_squared_error") 

    modelo.fit(treinamento_x, treinamento_y, batch_size=10, epochs=20)

    # Vriar dados de teste
    dados_teste = dados_entre_0_e_1[tamanho_dados_treinamento - 60:, :]

    teste_x = []
    teste_y = cotacao[tamanho_dados_treinamento: , :] 

    for i in range(60, len(dados_teste)):
        teste_x.append(dados_teste[i - 60: i, 0])

    # Reshape
    teste_x = np.array(teste_x)
    teste_x = teste_x.reshape(teste_x.shape[0], teste_x.shape[1], 1)

    # Pegando predições do modelo
    predicoes = modelo.predict(teste_x)

    # Tirando a escala dos dados
    predicoes = escalador.inverse_transform(predicoes)

    # Criando o grafico do modelo
    treinamento = acao.iloc[:tamanho_dados_treinamento, :]
    df_previsao = pd.DataFrame({"Date": acao['Date'].iloc[tamanho_dados_treinamento:],
                            "Price": acao['Price'].iloc[tamanho_dados_treinamento:],
                            "predicoes": predicoes.reshape(len(predicoes))})
    
    # Setar a data como index dos df (df_previsao e treinamento)
    df_previsao.set_index('Date', inplace=True)
    treinamento.set_index('Date', inplace=True)

    df_previsao.sort_index()

    df_previsao_semanal = df_previsao
    df_previsao_semanal['Semana'] = ((df_previsao_semanal.index - df_previsao_semanal.index.min()).days // 7) + 1

    df_previsao_semanal = df_previsao_semanal.reset_index() 


    df_previsao_semanal.reset_index() 
    # Agrupar por semana e calcular o preço médio semanal para real e previsões
    df_teste_semana = df_previsao_semanal.groupby('Semana').agg({
        'Price': 'mean',       # Preço real médio por semana
        'predicoes': 'mean',   # Previsão média por semana
        'Date': 'first'        # Pega a primeira data da semana 
    })

    # Adicionar colunas de mês e ano com base na coluna 'Data'
    df_teste_semana['Mes'] = df_teste_semana['Date'].dt.month
    df_teste_semana['Ano'] = df_teste_semana['Date'].dt.year

    # Calcular a variação percentual semanal para Price e Predicoes
    df_teste_semana['Variação Real (%)'] = df_teste_semana['Price'].pct_change() * 100
    df_teste_semana['Variação Prevista (%)'] = df_teste_semana['predicoes'].pct_change() * 100


    # Remover NaN (primeira linha não terá variação por não ter valor anterior)
    df_teste_semana = df_teste_semana.dropna()

    # Calcular se a direção do valor previsto foi igual ao real (1 = acertou | 0 = errou)
    df_teste_semana['direcao_correta'] = np.where(
        (df_teste_semana['Variação Real (%)'] > 0) & (df_teste_semana['Variação Prevista (%)'] > 0) |
        (df_teste_semana['Variação Real (%)'] < 0) & (df_teste_semana['Variação Prevista (%)'] < 0),
        1, 0
    )

    df_teste_semana.to_csv(f"data\previsão semanal - {nome}.csv")

    return df_teste_semana


if ano > 2021:
    with ThreadPoolExecutor() as executor:
        # Execute as 4 chamadas de função em paralelo
        executor.submit(previsao, df_bitcoin, 'Bitcoin')
        executor.submit(previsao, df_ethereum, 'Ethereum')
        executor.submit(previsao, df_bnb, 'BNB')
        executor.submit(previsao, df_solana, 'Solana')
else:
    # Se o ano for menor ou igual a 2021, remove o arquivo da solana se tiver e gera um novo dos outros
    if os.path.exists(caminho_arquivos / 'Solana.csv'):
        os.remove(caminho_arquivos / 'previsão semanal - Solana.csv')
    else: 
        pass
    
    with ThreadPoolExecutor() as executor:
        # Execute as 3 chamadas de função em paralelo
        executor.submit(previsao, df_bitcoin, 'Bitcoin')
        executor.submit(previsao, df_ethereum, 'Ethereum')
        executor.submit(previsao, df_bnb, 'BNB')
    

[array([0.00314642, 0.00347005, 0.00371428, 0.00524403, 0.00305502,
       0.00151028, 0.00155074, 0.00173053, 0.00171405, 0.00178297,
       0.        , 0.00037008, 0.00059033, 0.00051242, 0.00053639,
       0.00062779, 0.00171255, 0.00129752, 0.00164513, 0.00161067,
       0.00201371, 0.00199873, 0.00203918, 0.00162415, 0.00163314,
       0.00195078, 0.00198674, 0.00199423, 0.0019343 , 0.00202719,
       0.00269843, 0.00295164, 0.00327527, 0.00341012, 0.0036918 ,
       0.00345657, 0.00358542, 0.0039585 , 0.0040469 , 0.0029007 ,
       0.00317938, 0.0033397 , 0.0031629 , 0.00314642, 0.0033397 ,
       0.0033382 , 0.0036933 , 0.0039555 , 0.00399895, 0.00394801,
       0.00437802, 0.00499831, 0.00502079, 0.00579091, 0.00585983,
       0.0054493 , 0.00578642, 0.00604412, 0.00605162, 0.00655205])]
[0.007043488032363187]
[array([0.00314642, 0.00347005, 0.00371428, 0.00524403, 0.00305502,
       0.00151028, 0.00155074, 0.00173053, 0.00171405, 0.00178297,
       0.        , 0.00037008, 0.00

c:\Users\yupopic\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
Epoch 1/20
Epoch 1/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 22s 96ms/step - loss: 0.0110
125/214 ━━━━━━━━━━━━━━━━━━━━ 8s 92ms/step - loss: 0.0149Epoch 2/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 23s 83ms/step - loss: 0.0065
Epoch 2/20
214/214 ━━━━━━━━━━━━━━━━━━━━ 29s 86ms/step - loss: 0.0104
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 7s 81ms/step - loss: 0.0030
Epoch 3/20
214/214 ━━━━━━━━━━━━━━━━━━━━ 33s 89ms/step - loss: 0.0076
Epoch 2/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 7s 80ms/step - loss: 0.0016
Epoch 4/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 15s 79ms/step - loss: 0.0012
Epoch 3/20
214/214 ━━━━━━━━━━━━━━━━━━━━ 17s 79ms/step - loss: 0.0018
75/84 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 0.0012Epoch 3/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - loss: 0.0012
Epoch 5/20
214/214 ━━━━━━━━━━━━━━━━━━━━ 18s 85ms/step - loss: 0.0014
Epoch 3/20
182/182 ━━━━━━━━━━━━━━━━━━━━ 16s 88ms/step - loss: 9.1070e-04
 17/214 ━━━━━━━━━━━━━━━━━━━━ 20s 103ms/step - loss: 7.4198e-04Epoch 4/20
84/84 ━━━━━━━━━━━━━━━━━━━━ 8s 93ms/step - 